In [4]:
import os as os
import numpy as np
import pandas as pd
# import polars as pl
import tqdm
import slippi as slp
from slippi import Game
from slippi.parse import parse
from slippi.parse import ParseEvent
from joblib import Parallel, delayed
from multiprocessing import Manager
import time

In [5]:
# Set the number of time steps in the model inputs
frames_per_input = 60 * 12  # 12 seconds of gameplay

# Function to one-hot encode controller bitmask
def one_hot_encode(bitmask):
    labels = ['DPAD_LEFT', 'DPAD_RIGHT', 'DPAD_DOWN', 'DPAD_UP', 'Z', 'R', 'L', 'A', 'B', 'X', 'Y', 'START']
    encoded_values = [1, 2, 4, 8, 16, 32, 64, 256, 512, 1024, 2048, 4096]

    # Create a dictionary mapping labels to their encoded values
    label_to_value = dict(zip(labels, encoded_values))

    # Initialize a list to store the one-hot encoded values
    one_hot_encoded = [0] * len(labels)

    # Iterate through labels and set the corresponding one-hot encoded value
    for label, value in label_to_value.items():
        if bitmask & value:
            one_hot_encoded[labels.index(label)] = 1

    return one_hot_encoded

# Function to get frame data for a given set of frames and port
def get_inputs_of_all_frames(frames, port):
    all_frame_inputs = np.empty((len(frames), 18))  # Initialize an empty NumPy array    
    for i, frame in enumerate(frames):   # Takes all the frames, skipping the first 5 seconds.
        buttons = one_hot_encode(frame.ports[port].leader.pre.buttons.physical.value)
        j_x = frame.ports[port].leader.pre.joystick.x
        j_y = frame.ports[port].leader.pre.joystick.y
        c_x = frame.ports[port].leader.pre.cstick.x
        c_y = frame.ports[port].leader.pre.cstick.y
        t_l = frame.ports[port].leader.pre.triggers.physical.l
        t_r = frame.ports[port].leader.pre.triggers.physical.r

        frame_data = buttons + [j_x, j_y, c_x, c_y, t_l, t_r]
        all_frame_inputs[i] = frame_data

    return all_frame_inputs

# def get_inputs_of_all_frames_list(frames, port):
#     all_frame_inputs = [] # Initialize an empty NumPy array    
#     for i, frame in enumerate(frames):   # Takes all the frames, skipping the first 5 seconds.
#         all_frame_inputs += [one_hot_encode(frame.ports[port].leader.pre.buttons.physical.value) + [
#         frame.ports[port].leader.pre.joystick.x,
#         frame.ports[port].leader.pre.joystick.y,
#         frame.ports[port].leader.pre.cstick.x,
#         frame.ports[port].leader.pre.cstick.y,
#         frame.ports[port].leader.pre.triggers.physical.l,
#         frame.ports[port].leader.pre.triggers.physical.r
#         ]
#         ]
#         # frame_data = buttons + [j_x, j_y, c_x, c_y, t_l, t_r]
#         # all_frame_inputs[i] = frame_data

#     return all_frame_inputs

# Function to get frame data for a single frame and port
def get_inputs_of_frame(event):
    frame_inputs = one_hot_encode(event.leader.pre.buttons.physical.value) + [
    # frame_inputs = [
    # frame_inputs += [
        event.leader.pre.joystick.x,
        event.leader.pre.joystick.y,
        event.leader.pre.cstick.x,
        event.leader.pre.cstick.y,
        event.leader.pre.triggers.physical.l,
        event.leader.pre.triggers.physical.r
    ]
    return frame_inputs

# def get_inputs_of_frame_numpy(event,array):
#     array = one_hot_encode(event.leader.pre.buttons.physical.value) + [
#     # frame_inputs += [
#         event.leader.pre.joystick.x,
#         event.leader.pre.joystick.y,
#         event.leader.pre.cstick.x,
#         event.leader.pre.cstick.y,
#         event.leader.pre.triggers.physical.l,
#         event.leader.pre.triggers.physical.r
#     ]
#     return 

In [11]:
# dataset_path = './Slippi_Public_Dataset_v3/'
# slp_files = [file for file in os.listdir(dataset_path) if file.endswith('.slp') and 'Sheik' in file and 'Fox' in file]

# num_files = 100

def parse_game_event(dataset_path,slp_file):
    
    file_path = os.path.join(dataset_path, slp_file)

    sheik_port = None
    fox_port = None
    sheik_inputs = []
    fox_inputs = []
    is_cpu = False # is at least one player a cpu?
    last_frame = False 

    def handle_start(event):
        nonlocal sheik_port, fox_port, is_cpu  # Use nonlocal to declare that we're using these variables from the enclosing scope 
        
        # Find which port is Fox and which is Sheik
        occupied_ports = [i for i, port in enumerate(event.players) if port is not None]  
        if event.players[occupied_ports[0]].character.name == 'SHEIK':
            sheik_port = occupied_ports[0]
            fox_port = occupied_ports[1]
        else:
            sheik_port = occupied_ports[1]
            fox_port = occupied_ports[0]
        
        # Is one player a cpu?
        # event.players[sheik_port].type.value returns 0 if human and 1 if cpu
        # if event.players[sheik_port].type.value or event.players[fox_port].type.value:
        #     is_cpu = True
        
    def handle_frame(event):
        # Frame index starts at -123. The game shouts GO on frame 0.
        # if  event.index >= 0:
        #     nonlocal sheik_port, fox_port, last_frame  # Use nonlocal to declare that we're using these variables from the enclosing scope
            
        #     sheik_inputs.append(get_inputs_of_frame(event.ports[sheik_port]))
        #     fox_inputs.append(get_inputs_of_frame(event.ports[fox_port]))
        
        nonlocal sheik_port, fox_port, last_frame  # Use nonlocal to declare that we're using these variables from the enclosing scope
            
        sheik_inputs.append(get_inputs_of_frame(event.ports[sheik_port]))
        fox_inputs.append(get_inputs_of_frame(event.ports[fox_port]))
        
            # If we need to do something extra with the last frame (like figure out who won).
            # if last_frame:
            #     print('Last frame: ', event.index)
        
    # One frame is handled after the event END
    def handle_end(event):
        nonlocal last_frame
        last_frame = True
        
    
    handler = {
        # ParseEvent.START: handle_start,
        # ParseEvent.FRAME: handle_frame,
        # ParseEvent.END: handle_end
    }
    
    parse(file_path, handler)
    
    # We only care about games that don't have the cpu.
    # Before we append the game data to our list we need to check.
    # if not is_cpu:
    #     print('No cpu')


# t_1 = time.time()
# for slp_file in tqdm.tqdm(slp_files[:num_files]):
#     parse_game_event(dataset_path,slp_file)
#     # print()
# t_2 = time.time()

# time_event = t_2-t_1
# print(time_event)

In [12]:
parse_game_event('C:\\Users\\jaspa\\Grant ML\\slp\\data','diamond-diamond-351f2ab78a6cbc3170af66fc.slp')

ParseError: Parse error (C:\Users\jaspa\Grant ML\slp\data\diamond-diamond-351f2ab78a6cbc3170af66fc.slp @0x27826c): [Errno None] None: 'C:\\Users\\jaspa\\Grant ML\\slp\\data\\diamond-diamond-351f2ab78a6cbc3170af66fc.slp'

In [ ]:
dataset_path = './Slippi_Public_Dataset_v3/'
slp_files = [file for file in os.listdir(dataset_path) if file.endswith('.slp') and 'Sheik' in file and 'Fox' in file]

num_files = 10

def parse_game_object(dataset_path,slp_file):
    file_path = os.path.join(dataset_path, slp_file)
    game = Game(file_path)
    ports = [i for i, port in enumerate(game.start.players) if port is not None]
    port_1 = ports[0]
    port_2 = ports[1]
    frames = game.frames
    
    get_inputs_of_all_frames(frames, port_1)
    get_inputs_of_all_frames(frames, port_2)


# t_1 = time.time()
# for slp_file in slp_files[:num_files]:
#     parse_game_object(dataset_path,slp_file)
# t_2 = time.time()
# time_object = t_2-t_1
# print(time_object)

In [ ]:
# print(time_event/time_object)

In [ ]:
#### In what order are events handled? ####


# dataset_path = './Slippi_Public_Dataset_v3/'
# slp_files = [file for file in os.listdir(dataset_path) if file.endswith('.slp') and 'Sheik' in file and 'Fox' in file]

# num_files = 10

# def parse_game_event(dataset_path,slp_file):
    
#     file_path = os.path.join(dataset_path, slp_file)
    
#     # occupied_ports = []  # Global list to store results
#     player_characters = None
#     # frame_inputs_players = []
#     # player_1_inputs = []
#     # player_2_inputs = []
#     player_inputs = None
#     # global player_input_test=
#     global metadata_test 
#     global frame_test 
#     metadata_test = 0
#     frame_test = 0

#     def handle_metadata(event):
#         # game_length = event.duration
#         # global metadata_test
#         # # player_inputs = np.empty([game_length,2,18])
#         # # global player_input_test
#         # # player_input_test = game_length
#         # # player_input_test = np.empty(game_length)
#         # # player_2_inputs = np.empty(game_length,18)
#         # metadata_test += 1
#         print('metadata', event.duration)
    
#     def handle_metadata_raw(event):
#         print('metadata raw')
        
#     def handle_start(event):
#         print('start')

#     def handle_frame(event):
#         print('frame',event.index)
        
#     def handle_frame_start(event):
#         print('frame start', event.index)
        
#     def handle_frame_end(event):
#         print('frame start', event.index)
        
#     def handle_end(event):
#         print('end')
    
#     def handle_item(event):
#         print('item')
        


#     handler = {
        
#         ParseEvent.START: handle_start,
#         ParseEvent.FRAME: handle_frame,
#         ParseEvent.METADATA: handle_metadata,
#         ParseEvent.METADATA_RAW: handle_metadata_raw,
#         ParseEvent.FRAME_START: handle_frame_start,
#         ParseEvent.FRAME_END: handle_frame_end,
#         ParseEvent.END: handle_end,
#         ParseEvent.ITEM: handle_item
        
#     }
    
#     # handler_2 = {
    
#     # ParseEvent.START: handle_start,
#     # ParseEvent.FRAME: handle_frame,
#     # ParseEvent.METADATA: handle_metadata,
#     # ParseEvent.METADATA_RAW: handle_metadata_raw,
#     # ParseEvent.FRAME_START: handle_frame_start,
#     # ParseEvent.FRAME_END: handle_frame_end,

        
#     # }
    
#     parse(file_path, handler)
#     # parse(file_path, handler_2)
#     # print(metadata_test)
#     # print(frame_test)
    
# num_files = 1
# # t_1 = time.time()
# # # for slp_file in slp_files[:num_files]:
# # parse_game_event(dataset_path,'21_11_15 Falco + Captain Falcon (BF).slp')
# #     # print(player_input_test)
# # t_2 = time.time()

# t_1 = time.time()
# # for slp_file in slp_files[:num_files]:
# parse_game_event(dataset_path,'21_11_15 Falco + Captain Falcon (BF).slp')
#     # print(player_input_test)
# t_2 = time.time()

# time_event = t_2-t_1
# print(time_event)
# # print(results)

In [ ]:
def parse_game_event(dataset_path,slp_file,game_data_list):
    # print('.')
    try:
        file_path = os.path.join(dataset_path, slp_file)

        sheik_port = None
        fox_port = None
        sheik_inputs = []
        fox_inputs = []
        is_cpu = False # is at least one player a cpu?
        last_frame = False 

        def handle_start(event):
            nonlocal sheik_port, fox_port, is_cpu  # Use nonlocal to declare that we're using these variables from the enclosing scope 
            
            # Find which port is Fox and which is Sheik
            occupied_ports = [i for i, port in enumerate(event.players) if port is not None]  
            if event.players[occupied_ports[0]].character.name == 'SHEIK':
                sheik_port = occupied_ports[0]
                fox_port = occupied_ports[1]
            else:
                sheik_port = occupied_ports[1]
                fox_port = occupied_ports[0]
            
            # Is one player a cpu?
            # event.players[sheik_port].type.value returns 0 if human and 1 if cpu
            # if event.players[sheik_port].type.value or event.players[fox_port].type.value:
            #     is_cpu = True
            
        def handle_frame(event):
            # Frame index starts at -123. The game shouts GO on frame 0.
            # if   event.index >= 0:
            #     nonlocal sheik_port, fox_port, last_frame  # Use nonlocal to declare that we're using these variables from the enclosing scope
                
            #     sheik_inputs.append(get_inputs_of_frame(event.ports[sheik_port]))
            #     fox_inputs.append(get_inputs_of_frame(event.ports[fox_port]))
                
                # If we need to do something extra with the last frame (like figure out who won).
                # if last_frame:
                #     print('Last frame: ', event.index)
            nonlocal sheik_port, fox_port, last_frame  # Use nonlocal to declare that we're using these variables from the enclosing scope
            
            sheik_inputs.append(get_inputs_of_frame(event.ports[sheik_port]))
            fox_inputs.append(get_inputs_of_frame(event.ports[fox_port]))
            
        # One frame is handled after the event END
        def handle_end(event):
            nonlocal last_frame
            last_frame = True
            
        
        handler = {
            ParseEvent.START: handle_start,
            ParseEvent.FRAME: handle_frame,
            # ParseEvent.END: handle_end
        }
        
        parse(file_path, handler)
        
        game_data_list.append([sheik_inputs,fox_inputs])
        # We only care about games that don't have the cpu.
        # Before we append the game data to our list we need to check.
        # if not is_cpu:
        #     game_data_list.append([sheik_inputs,fox_inputs])
        # else:
        #     print('CPU is playing')

    except Exception as e:
        print(f"Error processing {slp_file}: {str(e)}")


def parse_batch(dataset_path,slp_batch, game_data_list):
    for slp_file in slp_batch:
        parse_game_event( dataset_path,slp_file, game_data_list)
        
# Helper function to create batches
def create_batches(slp_files, batch_size):
    for i in range(0, len(slp_files), batch_size):
        yield slp_files[i:i + batch_size]

       
# frames_per_input = 60 * 12     # 12 seconds of gameplay==
dataset_path = './Slippi_Public_Dataset_v3/'
slp_files = [file for file in os.listdir(dataset_path) if file.endswith('.slp') and 'Sheik' in file and 'Fox' in file]
num_files = 100
# batch_size = 50

# Create batches of slp_files
# slp_batches = list(create_batches(slp_files[:num_files], batch_size))
# print(slp_batches)

manager = Manager()
game_data_list = manager.list()

# Use joblib to parallelize processing of SLP files
# Parallel(n_jobs=1, verbose=10)(delayed(parse_game_event)( dataset_path,slp_file, game_data_list) for slp_file in tqdm.tqdm(slp_files[:num_files]))

# Parallel(n_jobs=-1, verbose=10)(delayed(parse_batch)( dataset_path,slp_batch, game_data_list) for slp_batch in tqdm.tqdm(slp_batches))
# 
# print(game_data_list)
# print(np.array(game_data_list).shape)

In [ ]:
def parse_game_event(dataset_path,slp_file,game_data_list):
    # print('.')
    try:
        file_path = os.path.join(dataset_path, slp_file)

        sheik_port = None
        fox_port = None
        sheik_inputs = []
        fox_inputs = []
        is_cpu = False # is at least one player a cpu?
        last_frame = False 

        def handle_start(event):
            nonlocal sheik_port, fox_port, is_cpu  # Use nonlocal to declare that we're using these variables from the enclosing scope 
            
            # Find which port is Fox and which is Sheik
            occupied_ports = [i for i, port in enumerate(event.players) if port is not None]  
            if event.players[occupied_ports[0]].character.name == 'SHEIK':
                sheik_port = occupied_ports[0]
                fox_port = occupied_ports[1]
            else:
                sheik_port = occupied_ports[1]
                fox_port = occupied_ports[0]
            
            # Is one player a cpu?
            # event.players[sheik_port].type.value returns 0 if human and 1 if cpu
            # if event.players[sheik_port].type.value or event.players[fox_port].type.value:
            #     is_cpu = True
            
        def handle_frame(event):
            # Frame index starts at -123. The game shouts GO on frame 0.
            if    event.index >= 0:
                nonlocal sheik_port, fox_port, last_frame  # Use nonlocal to declare that we're using these variables from the enclosing scope
                
                sheik_inputs.append(get_inputs_of_frame(event.ports[sheik_port]))
                fox_inputs.append(get_inputs_of_frame(event.ports[fox_port]))
                
            #     # If we need to do something extra with the last frame (like figure out who won).
            #     if last_frame:
            #         print('Last frame: ', event.index)
            # nonlocal sheik_port, fox_port, last_frame  # Use nonlocal to declare that we're using these variables from the enclosing scope
            
            # sheik_inputs.append(get_inputs_of_frame(event.ports[sheik_port]))
            # fox_inputs.append(get_inputs_of_frame(event.ports[fox_port]))
            
        # One frame is handled after the event END
        def handle_end(event):
            nonlocal last_frame
            last_frame = True
            
        
        handler = {
            ParseEvent.START: handle_start,
            ParseEvent.FRAME: handle_frame,
            ParseEvent.END: handle_end
        }
        
        parse(file_path, handler)
        # game_data_list.extend([sheik_inputs,fox_inputs])
        game_data_list.append([sheik_inputs,fox_inputs])
        # del sheik_inputs, sheik_port, fox_inputs, fox_port
        # We only care about games that don't have the cpu.
        # Before we append the game data to our list we need to check.
        # if not is_cpu:
        #     game_data_list.append([sheik_inputs,fox_inputs])
        # else:
        #     print('CPU is playing')

    except Exception as e:
        print(f"Error processing {slp_file}: {str(e)}")


# # def parse_batch(dataset_path,slp_batch, game_data_list):
# #     for slp_file in slp_batch:
# #         parse_game_event( dataset_path,slp_file, game_data_list)
        
# # # Helper function to create batches
# # def create_batches(slp_files, batch_size):
# #     for i in range(0, len(slp_files), batch_size):
# #         yield slp_files[i:i + batch_size]

       
# # frames_per_input = 60 * 12     # 12 seconds of gameplay==
# dataset_path = './Slippi_Public_Dataset_v3/'
# slp_files = [file for file in os.listdir(dataset_path) if file.endswith('.slp') and 'Sheik' in file and 'Fox' in file]
# num_files = 1000
# # batch_size = 10

# # Create batches of slp_files
# # slp_batches = list(create_batches(slp_files[:num_files], batch_size))
# # print(slp_batches)

# manager = Manager()
# game_data_list = manager.list()

# # Use joblib to parallelize processing of SLP files
# Parallel(n_jobs=-1, verbose=10)(delayed(parse_game_event)( dataset_path,slp_file, game_data_list) for slp_file in tqdm.tqdm(slp_files[:num_files]))

# # for slp_file in tqdm.tqdm(slp_files[:num_files]):
# #     parse_game_event( dataset_path,slp_file, game_data_list)
# # Parallel(n_jobs=-1, verbose=10)(delayed(parse_batch)( dataset_path,slp_batch, game_data_list) for slp_batch in tqdm.tqdm(slp_batches))
# # 
# # print(game_data_list)
# # print(np.array(game_data_list).shape)

In [ ]:
# print(game_data_list)
# print(np.array(game_data_list).shape)